In [ ]:
import pandas as pd
from pathlib import Path
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt

import sys
sys.path.append('./rtaUtils')

from rtaUtils import data_loading, paths, common

airport_list = ('LEBL','LPPT','LFPO','LFPG','EGLL','LPPR','EDDF','EBBR','EHAM','LEBB','LIRF',
                'EDDM','EGKK','LEVC','LSGG','LECO','LIMC','EIDW','LFML','LEZL','LOWW','LIPZ',
                'LIPE','LFLL','EDDL','EDDB','LTFM','LFMN','LFRS','LROP','EDDP','LGAV','EDDH',
                'LHBP','EKCH','EGCC','ELLX','LKPR','LIRN','LBSF','EPWA')

In [ ]:
pepe.columns

In [ ]:
pepe = data_loading.load_final_data('202201', 'train', 'EGLL', 30)
pepe[pepe.clouds>0]

# Airports

In [ ]:
month    = '20220[123456789]'
airports = ''

In [ ]:
airports_df = pd.read_csv(paths.airports_file_path)

In [ ]:
data = data_loading.load_clean_data(month, airports, ['aerodromeOfDeparture', 'fpId']).drop_duplicates()
data = data.groupby('aerodromeOfDeparture').count().reset_index()
data.columns = ['id', 'trajectories']

In [ ]:
joined = pd.merge(data, airports_df, on='id')

In [ ]:
import plotly.graph_objects as go
config = {
  'toImageButtonOptions': {
    'format': 'png', # one of png, svg, jpeg, webp
    'filename': 'airports',
    'height': 750,
    'width': 1000,
    'scale':2 # Multiply title/legend/axis/canvas sizes by this factor
  }
}
fig = px.scatter_mapbox(joined, 'lat', 'lon', zoom=3.8,width=1000, height=750,
                  mapbox_style="open-street-map", opacity = 0.8,
                  color='trajectories', 
                        labels={'trajectories':'Number of\ntrajectories'},
#                   size='trajectories',
#                   text='id'
                  # range_color=(50,300),
                  # color_continuous_scale='RdYlGn_r', #'bluered' 'Portland'
                  # color_continuous_midpoint=150,
                  # hover_data = {'MAE all':':.3f',
                  #               'RMSE all':':.3f'},
                  # hover_name='dataset'
                 )
fig.update_traces(marker={'size': 20})
fig.update_layout(coloraxis=dict(colorbar=dict(orientation='h', x=0.515, y=-0.12)))
fig.add_trace(go.Scattermapbox(
      mode = "markers", # include text mode
      lon = joined.lon,
      lat = joined.lat,
      text = joined.id,
    showlegend=False,
      marker = {'size': 3, 'color': 'white'},
      ))
fig.show(config=config)

# Number of operations

In [ ]:
data = data_loading.load_clean_data(month, airports, ['flightDate', 'fpId', 'aerodromeOfDeparture']).drop_duplicates()

In [ ]:
data['month'] = data.flightDate.apply(lambda x: x.strftime('%m-%Y'))
data = data.groupby(['month','aerodromeOfDeparture']).count().reset_index().drop('flightDate',axis=1)
data.columns = ['Month', 'Airport', 'Trajectories']

In [ ]:
px.bar(data, x='Month', y ='Trajectories', color='Airport')

# LEMD airport

In [ ]:
month    = '20220[2]'
airports = ''

In [ ]:
sort_stats = pd.read_csv('sort_stats.csv')
sort_stats = sort_stats[sort_stats.flightDate>'2022-01-31']

In [ ]:
flights = data_loading.load_clean_data(month, airports, ['fpId','aerodromeOfDeparture'])

In [ ]:
data = data_loading.load_clean_data(month, airports)
data = data[data.fpId.isin(flights.groupby('aerodromeOfDeparture').apply(lambda x: x.sample(10,random_state=42)).fpId)]
data = data[data.hav_distance < 120]

In [ ]:
config = {
  'toImageButtonOptions': {
    'format': 'png', # one of png, svg, jpeg, webp
    'filename': 'flights',
    'height': 750,
    'width': 1000,
    'scale':2 # Multiply title/legend/axis/canvas sizes by this factor
  }
}
fig = px.scatter_mapbox(data, 'latitude', 'longitude', zoom=8, width=1000, height=750,opacity = 1, 
                  mapbox_style="carto-positron", # open-street-map
                  # color='color', 
                        center=dict(lat=common.LEMD_LATITUDE, lon=common.LEMD_LONGITUDE+0.02),
                  #labels={'trajectories':'Number of\ntrajectories'},
                 )
fig.update_traces(marker={'size': 3})

fig.show(config=config)

In [ ]:
data = data_loading.load_clean_data(month, airports)
data = data[data.aerodromeOfDeparture.isin(['EDDF','EGKK','EHAM','LIMC','LTFM'])]
data = data[~data.fpId.isin(['AT06147029', 'AT06395033', 'AT06300603'])]
# data = data[data.hav_distance < 120]

In [ ]:
data2 = data[data.fpId.isin(data.groupby('aerodromeOfDeparture').apply(lambda x: x.sample(15,random_state=42)).fpId)]

config = {
  'toImageButtonOptions': {
    'format': 'png', # one of png, svg, jpeg, webp
    'filename': 'individualModels',
    'height': 750,
    'width': 1000,
    'scale':2 # Multiply title/legend/axis/canvas sizes by this factor
  }
}
fig = px.scatter_mapbox(data2, 'latitude', 'longitude', zoom=3, width=1000, height=750,#opacity = 1, 
                  mapbox_style="carto-positron", # open-street-map
                  color='aerodromeOfDeparture', 
                  # center=dict(lat=common.LEMD_LATITUDE, lon=common.LEMD_LONGITUDE+0.02),
                  hover_data=['fpId'],
                  #labels={'trajectories':'Number of\ntrajectories'},
                 )
fig.update_traces(marker={'size': 3})
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="right",
    x=0.99,
    title='Origin airport',
    bordercolor="black",
    borderwidth=1
))

fig.show(config=config)

In [ ]:
config = {
  'toImageButtonOptions': {
    'format': 'png', # one of png, svg, jpeg, webp
    'filename': 'flights',
    'height': 750,
    'width': 1000,
    'scale':2 # Multiply title/legend/axis/canvas sizes by this factor
  }
}
fig = px.scatter_mapbox(data2[data2.fpId == data2.fpId.iloc[-20]], 'latitude', 'longitude', zoom=8, width=1000, height=750,#opacity = 1, 
                  mapbox_style="open-street-map", # open-street-map
                  # color='color', 
                        center=dict(lat=common.LEMD_LATITUDE, lon=common.LEMD_LONGITUDE+0.02),
                     hover_data=['fpId']
                  #labels={'trajectories':'Number of\ntrajectories'},
                 )
fig.update_traces(marker={'size': 5})

fig.show(config=config)

# Model comparison

In [ ]:
data = pd.DataFrame([
# ['LSTMu10','MAE',150.06],
['LSTMu20\n(SP:30,LB:64)','MAE',159.24],
['LSTMu30\n(SP:60,LB:32)','MAE',161.17],
# ['LSTMu30','MAE',152.43],
['GBM\n(SP:30)','MAE',220.45],
['AB\n(SP:30)','MAE',222.41],
['RF\n(SP:30)','MAE',256.92],
# ['LSTMu10','RMSE',252.91],
['LSTMu20\n(SP:30,LB:64)','RMSE',257.82],
['LSTMu30\n(SP:60,LB:32)','RMSE',259.91],
# ['LSTMu30','RMSE',255.78],
['GBM\n(SP:30)','RMSE',323.73],
['AB\n(SP:30)','RMSE',458.28],
['RF\n(SP:30)','RMSE',552.01],
], columns=['Model','Metric','Value'])

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(6,3))
pepe = sns.barplot(data,x = 'Model', y='Value', hue='Metric', ax=ax, width=0.7, zorder=10)

ax.set_ylabel('Value (seconds)')
ax.set_xlabel(None)
plt.legend(ncol=2)
plt.tight_layout(pad=1.01)
plt.grid(axis='y', alpha=0.7, linestyle = '--', linewidth = 0.7)
plt.savefig('output/result1.pdf')
plt.show()

In [ ]:

data = pd.DataFrame(
[[30,16,8,10,223.03],
[30,16,8,20,245.87],
[30,16,8,30,233.20],
[30,32,16,10,224.28],
[30,32,16,20,220.07],
[30,32,16,30,233.20],
[30,64,32,10,162.90],
[30,64,32,20,159.24],
[30,64,32,30,161.04],
[30,128,64,10,184.00],
[30,128,64,20,184.50],
[30,128,64,30,180.59],
[60,16,16,10,218.77],
[60,16,16,20,218.80],
[60,16,16,30,211.56],
[60,32,32,10,164.18],
[60,32,32,20,163.60],
[60,32,32,30,161.17],
[60,64,64,10,150.06],
[60,64,64,20,149.74],
[60,64,64,30,152.43],], columns = ['Sampling','Lookback','Time','Units','MAE'])

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(9.5,3),sharey=True)
labels = ['Units','Sampling','Lookback']

for idx, a in enumerate(ax):
    sns.barplot(data, x = 'Time', y='MAE', hue=labels[idx],  ax=ax[idx], zorder=10,errorbar=None)
    a.grid(axis='y', alpha=0.7, linestyle = '--', linewidth = 0.7)
    a.legend(title = labels[idx], ncol = 5, fancybox=True, loc='lower center', bbox_to_anchor=(0.5, 1.0),
             title_fontsize ='small', fontsize='small',)
plt.tight_layout(pad=1.01)
plt.savefig('output/result2.pdf')

# Vectores a X tiempo vista

In [ ]:
import pandas as pd
from pathlib import Path
import plotly.express as px

import sys
sys.path.append('./rtaUtils')

from rtaUtils import data_loading, paths, common

airport_list = ('LEBL','LPPT','LFPO','LFPG','EGLL','LPPR','EDDF','EBBR','EHAM','LEBB','LIRF',
                'EDDM','EGKK','LEVC','LSGG','LECO','LIMC','EIDW','LFML','LEZL','LOWW','LIPZ',
                'LIPE','LFLL','EDDL','EDDB','LTFM','LFMN','LFRS','LROP','EDDP','LGAV','EDDH',
                'LHBP','EKCH','EGCC','ELLX','LKPR','LIRN','LBSF','EPWA')

In [ ]:
month    = '20220*'
airports = ''

In [ ]:
data = data_loading.load_clean_data(month, airports, ['fpId', 'vectorId', 'latitude','longitude','hav_distance', 'RTA', 'aerodromeOfDeparture'])
data = data.sort_values(by=['fpId','hav_distance'], ascending=(True,False))

In [ ]:
vectors = data[(data.hav_distance > 60)].groupby('fpId').nth(-1).reset_index()[['fpId','vectorId']]
vectors = pd.merge(data, vectors, on=['fpId','vectorId'])

In [ ]:
distance = 80
pepe = vectors[vectors.hav_distance>distance]
pepe = data[data.fpId.isin(pepe.fpId)].copy()
pepe = pepe[(pepe.hav_distance > 60)&(pepe.hav_distance <250)]

In [ ]:
fig = px.scatter_mapbox(pepe, 'latitude', 'longitude', zoom=8, width=1000, height=750,opacity = 1, 
                  mapbox_style="carto-positron", # open-street-map
                  color='aerodromeOfDeparture', 
                        center=dict(lat=common.LEMD_LATITUDE, lon=common.LEMD_LONGITUDE+0.02),
                  #labels={'trajectories':'Number of\ntrajectories'},
                        hover_data=['RTA', 'fpId']
                 )
fig.show()